In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("Done!")

Done!


In [ ]:
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

## Below, I have stripped the td table elements to get the values, put them into 3 separate lists, and then put them into a data frame (not before changing the Neighborhood to match the Borough if the Neighborhood has a value but Borough does not).

## I then removed the Not Assigned Borough rows and then did a self join on the list of Neighborhoods made after grouping them. 

In [88]:
mytable = soup.find('table')
tds = mytable.find_all('td')
entries = [td.text.strip() for td in tds]
post = []
borough = []
neighborhood = []
for i in range(0,len(entries)):
    if i%3 == 0:
               post.append(entries[i])
    elif i%3 == 1:
               borough.append(entries[i])
    elif i%3 == 2:
               neighborhood.append(entries[i])
for i in range(0,len(post)-1):
    if borough[i] != "Not assigned" and neighborhood[i] == "Not assigned":
        neighborhood[i] = borough[i]
wiki = pd.DataFrame({'Postcode':post,'Borough':borough,'Neighbourhood':neighborhood})
wiki = wiki[wiki["Borough"] != "Not assigned"]
wikijoin = pd.DataFrame(wiki.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index())
print(wikijoin.head(10))
wikijoin.shape

  Postcode      Borough                                  Neighbourhood
0      M1B  Scarborough                                  Rouge,Malvern
1      M1C  Scarborough           Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough                Guildwood,Morningside,West Hill
3      M1G  Scarborough                                         Woburn
4      M1H  Scarborough                                      Cedarbrae
5      M1J  Scarborough                            Scarborough Village
6      M1K  Scarborough      East Birchmount Park,Ionview,Kennedy Park
7      M1L  Scarborough                  Clairlea,Golden Mile,Oakridge
8      M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West
9      M1N  Scarborough                     Birch Cliff,Cliffside West


(103, 3)

### There should be 103 unique Postcode-Borough combinations.

In [74]:
geocodes = pd.read_csv('https://cocl.us/Geospatial_data')

## I decided to use the CSV since the geocodes package was not working. Below, I did a join by using the post codes as indices, then bringing it back as a named column

In [89]:
wikijoin = wikijoin.set_index("Postcode")
wikijoin = wikijoin.join(geocodes.set_index("Postal Code"))
wikijoin = wikijoin.reset_index()
wikijoin.rename(columns={"index":"Postcode"}, inplace=True)
wikijoin.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
